In [2]:
import torchvision.datasets as datasets
import torchvision.transforms as transforms

# Setup data directory
import pathlib
data_dir = pathlib.Path("../food_data")

In [3]:
# Get random 10% of training images
import random

# Setup data paths
data_path = data_dir / "food-101" / "images"
target_classes = ['pizza','steak','sushi']

# Change amount of data to get (e.g. 0.1 = random 10%, 0.2 = random 20%)
amount_to_get = 0.5

# Create function to separate a random amount of data
def get_subset(image_path=data_path,
                data_splits=["train", "test"], 
                target_classes=target_classes,
                amount=0.5,
                seed=42):
    random.seed(42)
    label_splits = {}
    
    # Get labels
    for data_split in data_splits:
        print(f"[INFO] Creating image split for: {data_split}...")
        label_path = data_dir / "food-101" / "meta" / f"{data_split}.txt"
        with open(label_path, "r") as f:
            labels = [line.strip("\n") for line in f.readlines() if line.split("/")[0] in target_classes] 
        
        # Get random subset of target classes image ID's
        number_to_sample = round(amount * len(labels))
        print(f"[INFO] Getting random subset of {number_to_sample} images for {data_split}...")
        sampled_images = random.sample(labels, k=number_to_sample)
        
        # Apply full paths
        image_paths = [pathlib.Path(str(image_path / sample_image) + ".jpg") for sample_image in sampled_images]
        label_splits[data_split] = image_paths
    return label_splits
        
label_splits = get_subset(amount=amount_to_get)
label_splits["train"][:10]

[INFO] Creating image split for: train...
[INFO] Getting random subset of 1125 images for train...
[INFO] Creating image split for: test...
[INFO] Getting random subset of 375 images for test...


[WindowsPath('../food_data/food-101/images/pizza/3269634.jpg'),
 WindowsPath('../food_data/food-101/images/pizza/1524655.jpg'),
 WindowsPath('../food_data/food-101/images/steak/2825100.jpg'),
 WindowsPath('../food_data/food-101/images/steak/225990.jpg'),
 WindowsPath('../food_data/food-101/images/steak/1839481.jpg'),
 WindowsPath('../food_data/food-101/images/pizza/38349.jpg'),
 WindowsPath('../food_data/food-101/images/pizza/3018077.jpg'),
 WindowsPath('../food_data/food-101/images/sushi/93139.jpg'),
 WindowsPath('../food_data/food-101/images/pizza/2702825.jpg'),
 WindowsPath('../food_data/food-101/images/sushi/200025.jpg')]

In [ ]:
# Create target directory path
target_dir_name = f"../data/50_food_images{str(int(amount_to_get*100))}_percent"
print(f"Creating directory: '{target_dir_name}'")

# Setup the directories
target_dir = pathlib.Path(target_dir_name)

# Make the directories
target_dir.mkdir(parents=True, exist_ok=True)

In [ ]:
import shutil

for image_split in label_splits.keys():
    for image_path in label_splits[str(image_split)]:
        dest_dir = target_dir / image_split / image_path.parent.stem / image_path.name
        if not dest_dir.parent.is_dir():
            dest_dir.parent.mkdir(parents=True, exist_ok=True)
        print(f"[INFO] Copying {image_path} to {dest_dir}...")
        shutil.copy2(image_path, dest_dir)

In [6]:
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import pathlib
import random
import shutil

class FoodDataOrganizer:
    def __init__(self, data_dir, target_classes, seed=42):
        self.data_dir = pathlib.Path(data_dir)
        self.image_path = self.data_dir / "food-101" / "images"
        self.target_classes = target_classes
        self.seed = seed

    def _get_image_labels(self, data_split):
        label_path = self.data_dir / "food-101" / "meta" / f"{data_split}.txt"
        with open(label_path, "r") as f:
            labels = [line.strip("\n") for line in f.readlines() if line.split("/")[0] in self.target_classes]
        return labels

    def get_random_subset(self, data_splits=["train", "test"], amount=1.0):
        random.seed(self.seed)
        label_splits = {}

        for data_split in data_splits:
            print(f"[INFO] Creating image split for: {data_split}...")
            labels = self._get_image_labels(data_split)

            number_to_sample = round(amount * len(labels))
            print(f"[INFO] Getting random subset of {number_to_sample} images for {data_split}...")
            sampled_images = random.sample(labels, k=number_to_sample)

            image_paths = [pathlib.Path(str(self.image_path / sample_image) + ".jpg") 
                            for sample_image in sampled_images]
            label_splits[data_split] = image_paths

        return label_splits

    def organize_subset(self, label_splits, target_dir_name):
        target_dir = pathlib.Path(target_dir_name)
        print(f"Creating directory: '{target_dir_name}'")

        target_dir.mkdir(parents=True, exist_ok=True)

        for image_split in label_splits.keys():
            for image_path in label_splits[str(image_split)]:
                dest_dir = target_dir / image_split / image_path.parent.stem / image_path.name
                if not dest_dir.parent.is_dir():
                    dest_dir.parent.mkdir(parents=True, exist_ok=True)
                print(f"[INFO] Copying {image_path} to {dest_dir}...")
                shutil.copy2(image_path, dest_dir)

if __name__ == "__main__":
    # Example usage
    data_dir = "../food_data" 
    final_list = ['pizza', 'steak', 'sushi'] 
    amount_to_get  = 0.4 
    target_dir_name = f"../data/50_food_images{str(int(amount_to_get*100))}_percent"

    organizer = FoodDataOrganizer(data_dir, final_list)
    label_splits = organizer.get_random_subset(amount=amount_to_get) 
    organizer.organize_subset(label_splits, target_dir_name)


[INFO] Creating image split for: train...
[INFO] Getting random subset of 900 images for train...
[INFO] Creating image split for: test...
[INFO] Getting random subset of 300 images for test...
Creating directory: '../data/50_food_images40_percent'
[INFO] Copying ..\food_data\food-101\images\pizza\3269634.jpg to ..\data\50_food_images40_percent\train\pizza\3269634.jpg...
[INFO] Copying ..\food_data\food-101\images\pizza\1524655.jpg to ..\data\50_food_images40_percent\train\pizza\1524655.jpg...
[INFO] Copying ..\food_data\food-101\images\steak\2825100.jpg to ..\data\50_food_images40_percent\train\steak\2825100.jpg...
[INFO] Copying ..\food_data\food-101\images\steak\225990.jpg to ..\data\50_food_images40_percent\train\steak\225990.jpg...
[INFO] Copying ..\food_data\food-101\images\steak\1839481.jpg to ..\data\50_food_images40_percent\train\steak\1839481.jpg...
[INFO] Copying ..\food_data\food-101\images\pizza\38349.jpg to ..\data\50_food_images40_percent\train\pizza\38349.jpg...
[INFO] 

## Data Transformations
1. Importing all the libraries

In [7]:
import os
import torch
import time
import copy
import torchvision
import torchvision.models as models
import numpy as np
from torchvision import datasets
from torch import nn
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder

* Data Transformations, ImageFolder creation, DataLoader creation.

In [10]:
def get_data_loaders(data_dir, mean, std, batch_size):
        data_transforms = {
            'train': transforms.Compose([
                transforms.RandomResizedCrop(224),
                transforms.RandomHorizontalFlip(),
                transforms.ToTensor(),
                transforms.Normalize(mean, std)
            ]),
            'val': transforms.Compose([
                transforms.Resize(256),
                transforms.CenterCrop(224),
                transforms.ToTensor(),
                transforms.Normalize(mean, std)
            ]),
        }

        image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x])
                            for x in ['train', 'val']}
        dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size, 
                                                        shuffle=True, num_workers=2)
                        for x in ['train', 'val']}
        dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
        class_names = image_datasets['train'].classes

        return dataloaders, dataset_sizes, class_names

mean = np.array([0.5, 0.5, 0.5])
std = np.array([0.25, 0.25, 0.25])
batch_size = 32
data_dir = "C:/datascienceprojects/food_image_classification/data/50_food_images40_percent"
get_data_loaders(data_dir = data_dir, mean=mean, std=std, batch_size=batch_size)

({'train': <torch.utils.data.dataloader.DataLoader at 0x164b4354980>,
  'val': <torch.utils.data.dataloader.DataLoader at 0x164a4d4e8d0>},
 {'train': 900, 'val': 300},
 ['pizza', 'steak', 'sushi'])

## Building the Model from scratch